Import dependencies and print Tensorflow version:

In [1]:
import tensorflow as tf
import numpy as np

tf.__version__

'2.3.0-rc2'

In [2]:
INPUT_SIZE = 299
INPUT_CHANNELS_COUNT = 3
CLASSES_COUNT = 10
DATASET_LENGTH = 100
BATCH_SIZE = 32

def get_empty_image(i):
    image = tf.zeros([INPUT_SIZE, INPUT_SIZE, INPUT_CHANNELS_COUNT])
    one_hot_label = np.zeros([CLASSES_COUNT], np.bool)
    one_hot_label[0] = 1
    return image, one_hot_label

dataset = tf.data.Dataset.range(DATASET_LENGTH).map(get_empty_image).batch(BATCH_SIZE)
dataset

<BatchDataset shapes: ((None, 299, 299, 3), (None, 10)), types: (tf.float32, tf.bool)>

In [3]:
base_model = tf.keras.applications.Xception(
    input_shape=(INPUT_SIZE, INPUT_SIZE, INPUT_CHANNELS_COUNT),
    weights='imagenet', include_top=False)
base_model.trainable = False

In [6]:
model = tf.keras.models.Sequential([
        base_model,
        tf.keras.layers.GlobalAveragePooling2D(),
        tf.keras.layers.Dense(CLASSES_COUNT, activation="relu")
    ])
model.compile(loss="categorical_crossentropy")
model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
xception (Functional)        (None, 10, 10, 2048)      20861480  
_________________________________________________________________
global_average_pooling2d_1 ( (None, 2048)              0         
_________________________________________________________________
dense_1 (Dense)              (None, 10)                20490     
Total params: 20,881,970
Trainable params: 20,490
Non-trainable params: 20,861,480
_________________________________________________________________


In [7]:
SAVED_MODEL_PATH = "saved_model"
model.save(SAVED_MODEL_PATH)

Instructions for updating:
This property should not be used in TensorFlow 2.0, as updates are applied automatically.
Instructions for updating:
This property should not be used in TensorFlow 2.0, as updates are applied automatically.
INFO:tensorflow:Assets written to: saved_model/assets


In [8]:
H5_SAVE_PATH = "saved_model.h5"
model.save(H5_SAVE_PATH, save_format="h5")

Load the Saved Model, disable training of its first layer (which is the Xception model) and print its summary to verify, that it is similar to the original model:

In [9]:
saved_model = tf.keras.models.load_model(SAVED_MODEL_PATH)
saved_model.layers[0].trainable = False
saved_model.summary(line_length=120)

Model: "sequential_1"
________________________________________________________________________________________________________________________
Layer (type)                                          Output Shape                                    Param #           
xception (Functional)                                 (None, 10, 10, 2048)                            20861480          
________________________________________________________________________________________________________________________
global_average_pooling2d_1 (GlobalAveragePooling2D)   (None, 2048)                                    0                 
________________________________________________________________________________________________________________________
dense_1 (Dense)                                       (None, 10)                                      20490             
Total params: 20,881,970
Trainable params: 20,490
Non-trainable params: 20,861,480
________________________________________________

And same for the Keras H5 model:

In [10]:
saved_model_h5 = tf.keras.models.load_model(H5_SAVE_PATH)
saved_model_h5.layers[0].trainable = False
saved_model_h5.summary(line_length=120)

Model: "sequential_1"
________________________________________________________________________________________________________________________
Layer (type)                                          Output Shape                                    Param #           
xception (Functional)                                 (None, 10, 10, 2048)                            20861480          
________________________________________________________________________________________________________________________
global_average_pooling2d_1 (GlobalAveragePooling2D)   (None, 2048)                                    0                 
________________________________________________________________________________________________________________________
dense_1 (Dense)                                       (None, 10)                                      20490             
Total params: 20,881,970
Trainable params: 20,490
Non-trainable params: 20,861,480
________________________________________________

Now when trying to fit the Saved Model, we will get ResourceExhaustedError (out of memory):

In [11]:
saved_model.fit(dataset, epochs=1)

ResourceExhaustedError:  OOM when allocating tensor with shape[728,728,1,1] and type float on /job:localhost/replica:0/task:0/device:GPU:0 by allocator GPU_0_bfc
	 [[node sequential_1/xception/block10_sepconv2/separable_conv2d (defined at <ipython-input-11-b4c664a47161>:1) ]]
Hint: If you want to see a list of allocated tensors when OOM happens, add report_tensor_allocations_upon_oom to RunOptions for current allocation info.
 [Op:__inference_train_function_79114]

Function call stack:
train_function


However, with the original model everything is alright:

In [12]:
model.fit(dataset, epochs=1)

4/4 [==============================] - 0s 112ms/step - loss: 16.1181


As well as with the Keras H5 model:

In [13]:
saved_model_h5.fit(dataset, epochs=1)

4/4 [==============================] - 0s 83ms/step - loss: 16.1181


We can even create a "copy" of the saved model by putting its layers into the new model and training will run without errors:

In [14]:
saved_model_copy = tf.keras.models.Sequential(saved_model.layers)
saved_model_copy.compile(loss="categorical_crossentropy")
saved_model_copy.fit(dataset, epochs=1)

4/4 [==============================] - 0s 82ms/step - loss: 16.1181
